In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp gdrive/My\ Drive/Colab\ Notebooks/my_txtutils.py my_txtutils.py
import my_txtutils as txt

In [3]:
import tensorflow as tf

!pip install -q keras
import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.12.0


Using TensorFlow backend.


In [0]:
# Define some variables
sequence_length = 30
batch_size = 200
alphabet_size = 98
internal_size = 512
#   -> Number of LSTM layers: 3
learning=rate = 0.001   # fixed learning rate
dropout_pkeep = 0.8   # some dropout

In [9]:
# Load Shakespeare data
shakedir = 'gdrive/My Drive/Colab Notebooks/shakespeare/*.txt'
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)

# Display some stats on the data
epoch_size = len(codetext) // (batch_size * sequence_length)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

Loading file gdrive/My Drive/Colab Notebooks/shakespeare/2kinghenryiv.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/cymbeline.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/tempest.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/merrywivesofwindsor.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/loveslabourslost.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/various.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/twogentlemenofverona.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/coriolanus.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/merchantofvenice.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/rapeoflucrece.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/allswellthatendswell.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/juliuscaesar.txt
Loading file gdrive/My Drive/Colab Notebooks/shakespeare/sonnets.txt
Loading file gdrive/My Dri

In [0]:
from keras.utils import to_categorical

# One-hot encode the coded text
codetext_prime = to_categorical(codetext, num_classes=alphabet_size)
valitext_prime = to_categorical(valitext, num_classes=alphabet_size)

In [0]:
X = []
y = []
for i in range(0, len(codetext_prime) - 1):
    sequence = codetext_prime[i:i + sequence_length + 1]
    if len(sequence) >= sequence_length + 1:
        X.append(sequence[0:sequence_length])
        y.append(sequence[1:sequence_length + 1])

In [29]:
model = keras.Sequential([
    keras.layers.GRU(internal_size, return_sequences=True, input_shape=(sequence_length, alphabet_size)),
    keras.layers.TimeDistributed(keras.layers.Dense(alphabet_size, activation='softmax'))
])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 30, 512)           938496    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 98)            50274     
Total params: 988,770
Trainable params: 988,770
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Callback for saving checkpoints
filepath="shakespeare-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [0]:
model.fit(X, y, epochs=40, batch_size=batch_size, callbacks=[checkpoint])